# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler # conda install -c conda-forge imbalanced-learn

df_customer = pd.read_csv("./data/customer.csv")
df_category = pd.read_csv("./data/category.csv")
df_product = pd.read_csv("./data/product.csv")
df_receipt = pd.read_csv("./data/receipt.csv")
df_store = pd.read_csv("./data/store.csv")
df_geocode = pd.read_csv("./data/geocode.csv")

/Users/miwa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/Users/miwa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 演習問題

---
> P-051: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"日"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"日"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [4]:
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']],
           pd.to_datetime(df_receipt['sales_epoch'], unit='s').dt.strftime('%d')],axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,03
1,1132,2,18
2,1102,1,12
3,1132,1,05
4,1102,2,21
5,1112,1,05
6,1102,2,05
7,1102,1,22
8,1112,2,04
9,1102,1,10


---
> P-052: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2000円以下を0、2000円超を1に2値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [6]:
df_tmp = df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id').amount.sum().reset_index()
df_tmp['amount_num'] = df_tmp['amount'].apply(lambda x: 0 if x <= 2000 else 1)
df_tmp.head(10)

,customer_id,amount,amount_num
0,CS001113000004,1298,0
1,CS001114000005,626,0
2,CS001115000010,3044,1
3,CS001205000004,1988,0
4,CS001205000006,3337,1
5,CS001211000025,456,0
6,CS001212000027,448,0
7,CS001212000031,296,0
8,CS001212000046,228,0
9,CS001212000070,456,0


---
> P-053: 顧客データフレーム（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に２値化せよ。さらにレシート明細データフレーム（df_receipt）と結合し、全期間において買い物実績のある顧客数を、作成した2値ごとにカウントせよ。

In [18]:
df_tmp = pd.concat([df_customer['customer_id'], df_customer['postal_cd'].apply(lambda x: 1 if int(x[:3]) <= 209 else 0)], axis=1)
df_tmp
pd.merge(df_receipt, df_tmp, on='customer_id', how='left')

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,postal_cd
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158,0.0
1,20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81,1.0
2,20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170,0.0
3,20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25,NaN
4,20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90,0.0
5,20190605,1275696000,S13003,1112,1,CS003515000195,P050102002,1,138,1.0
6,20181205,1259971200,S14024,1102,2,CS024514000042,P080101005,1,30,0.0
7,20190922,1285113600,S14040,1102,1,CS040415000178,P070501004,1,128,0.0
8,20170504,1209859200,S13020,1112,2,ZZ000000000000,P071302010,1,770,NaN
9,20191010,1286668800,S14027,1102,1,CS027514000015,P071101003,1,680,0.0


---
> P-054: 顧客データデータフレーム（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに抽出せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。結果は10件表示させれば良い。

In [51]:
def convert_prefectures(x):
    if  x == '埼玉県':
        return 11
    elif x == '千葉県':
        return 12
    elif x == '東京都':
        return 13
    else:
        return 14

df_tmp = df_customer['address'].str[:3].apply(convert_prefectures).reset_index()
df_tmp = pd.concat([df_customer[['customer_id', 'address']], df_customer['address'].str[:3].apply(convert_prefectures).reset_index()], axis=1)
df_tmp.head(10)

,customer_id,address,index,address
0,CS021313000114,神奈川県伊勢原市粟窪**********,0,14
1,CS037613000071,東京都江東区南砂**********,1,13
2,CS031415000172,東京都渋谷区代々木**********,2,13
3,CS028811000001,神奈川県横浜市泉区和泉町**********,3,14
4,CS001215000145,東京都大田区仲六郷**********,4,13
5,CS020401000016,東京都板橋区若木**********,5,13
6,CS015414000103,東京都江東区北砂**********,6,13
7,CS029403000008,千葉県浦安市海楽**********,7,12
8,CS015804000004,東京都江東区北砂**********,8,13
9,CS033513000180,神奈川県横浜市旭区善部町**********,9,14


---
> P-055: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額と合計ともに表示せよ。カテゴリ値は上から順に1〜4とする。結果は10件表示させれば良い。
>
> - 最小値以上第一四分位未満
> - 第一四分位以上第二四分位未満
> - 第二四分位以上第三四分位未満
> - 第三四分位以上

In [79]:
categories = df_receipt.groupby('customer_id').amount.sum().quantile([0, 0.25, 0.5, 0.75, 1.0]).reset_index().amount

def label_amount(x):
    if x < categories[0] :
        return 1
    elif categories[0] <= x < categories[1]:
        return 2
    elif categories[1] <= x < categories[2]:
        return 3
    else:
        return 4

df_tmp = df_receipt.groupby('customer_id').amount.sum().reset_index()
df_tmp['amount_label'] = df_tmp.amount.apply(label_amount)
df_tmp.head(10)

,customer_id,amount,amount_label
0,CS001113000004,1298,3
1,CS001114000005,626,3
2,CS001115000010,3044,4
3,CS001205000004,1988,4
4,CS001205000006,3337,4
5,CS001211000025,456,2
6,CS001212000027,448,2
7,CS001212000031,296,2
8,CS001212000046,228,2
9,CS001212000070,456,2
